导入库

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import time
import datetime
from sklearn.model_selection import train_test_split

import cv2

import h5py
%matplotlib inline

random_state = 424
dataset_path = 'dataset'

这个数据集是26位司机的各种样子的照片。所以我按照司机来划分训练集和验证集。

In [6]:
# 读取csv文件
dataset = pd.read_csv('dataset/driver_imgs_list.csv')

# 将前21为司机的照片作为训练集，后五位司机的照片作为测试集
drive_id_train = dataset['subject'].unique()[:-1]
drive_id_test = dataset['subject'].unique()[-1:]

# 组合路径
dataset['path']=dataset[['classname', 'img']].apply(lambda x: '/'.join(x), axis=1)

# 将路径转换成list格式
train_path = dataset['path'].loc[dataset['subject'].isin(drive_id_train)].tolist()
test_path = dataset['path'].loc[dataset['subject'].isin(drive_id_test)].tolist()

In [7]:
import os
import shutil


if os.path.exists('dataset/train2'):
    print('split dataset exist')
else:
    os.mkdir('dataset/train2')
    os.mkdir('dataset/valid2')
    for i in range(10):
        os.mkdir('dataset/train2/c'+str(i))
        os.mkdir('dataset/valid2/c'+str(i))
    # 复制图片到新的数据集
    if os.path.exists('dataset/train2'):
        print('train dataset copy begain')
        for filename in tqdm(train_path):
            shutil.copyfile('dataset/train/'+filename, 'dataset/train2/'+filename)
    if os.path.exists('dataset/valid2'):
        print('test dataset copy begain')
        for filename in tqdm(test_path):
            shutil.copyfile('dataset/train/'+filename, 'dataset/valid2/'+filename)




  0%|          | 0/21601 [00:00<?, ?it/s]

train dataset copy begain


  0%|          | 0/823 [00:00<?, ?it/s]

test dataset copy begain


100%|██████████| 823/823 [00:01<00:00, 811.91it/s]


预训练模型模板

In [8]:
from keras.models import Sequential, Model
from keras.applications import *
from keras.layers import Dropout, Flatten, Dense, Input, GlobalAveragePooling2D
from keras.optimizers import *
from keras.preprocessing.image import *
from keras.callbacks import ModelCheckpoint

train_dir = 'dataset/train2'  # 训练集数据
val_dir = 'dataset/valid2' # 验证集数据
nb_classes = len(glob.glob(train_dir + "/*"))  # 分类数


batch_size = 128
epochs = 5
# def VGG16_model(X_train, y_train, X_test, y_test):
def run_a_model(BASE_MODEL,input_shape, fine_tune_layer, preprocessing=None, model_name_option=''):
    input_height = input_shape[0]
    input_width = input_shape[1]
    input_tensor = Input((input_height, input_width, 3))
    base_model = BASE_MODEL(input_tensor=Input((input_height, input_width, 3)),
                            weights='imagenet', 
                            include_top=False, 
                            input_shape=(input_height, input_width, 3)
                           )
    
    
    x = input_tensor
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(10, activation='softmax')(x)
    model = Model(base_model.input, x)    
    print("total layer count {}".format(len(base_model.layers)))
    
    # 冻结基础层
    '''
    for layer in base_model.layers:
        layer.trainable = False
    '''
    for i in range(fine_tune_layer):
        model.layers[i].trainable = False
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    train_gen = ImageDataGenerator(
        rotation_range=10.,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.1,
        zoom_range=0.1,
        preprocessing_function=preprocessing
    )
    
    test_gen = ImageDataGenerator(
        preprocessing_function=preprocessing
    )
    
    train_generator = train_gen.flow_from_directory(train_dir, (input_height, input_width), 
                                                    shuffle=True, batch_size=batch_size, class_mode='categorical')
    test_generator = test_gen.flow_from_directory(val_dir, (input_height, input_width), 
                                                  shuffle=True, batch_size=batch_size, class_mode='categorical')
    
    
    steps_train_sample = train_generator.samples // batch_size + 1
    steps_valid_sample = test_generator.samples // batch_size + 1

    model.fit_generator(
        train_generator,
        steps_per_epoch=steps_train_sample,
        epochs=5,
        validation_data=test_generator,
        validation_steps=steps_valid_sample)
    
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.00001), metrics=['accuracy'])
    model.fit_generator(
        train_generator,
        steps_per_epoch=steps_train_sample,
        epochs=5,
        validation_data=test_generator,
        validation_steps=steps_valid_sample)
    
    
    model.save("models/model_{}{}.h5".format(BASE_MODEL.func_name, model_name_option))

    return

In [29]:
# VGG16_model(X_train, y_train, X_test, y_test)
run_a_model(VGG16, (224, 224), 18, optimizer='adam', preprocessing=None, model_name_option='')

total layer count 19
Found 20787 images belonging to 10 classes.
Found 1637 images belonging to 10 classes.
Epoch 1/5
163/163 [==============================] - 337s - loss: 9.9453 - acc: 0.1479 - val_loss: 5.2382 - val_acc: 0.2297
Epoch 2/5
163/163 [==============================] - 319s - loss: 5.7910 - acc: 0.2895 - val_loss: 3.8723 - val_acc: 0.2999
Epoch 3/5
163/163 [==============================] - 319s - loss: 3.1380 - acc: 0.4014 - val_loss: 3.0632 - val_acc: 0.3103
Epoch 4/5
163/163 [==============================] - 319s - loss: 2.0870 - acc: 0.4584 - val_loss: 2.0766 - val_acc: 0.3775
Epoch 5/5
163/163 [==============================] - 319s - loss: 1.7443 - acc: 0.4817 - val_loss: 2.2278 - val_acc: 0.3445


In [3]:
# ResNet50
run_a_model(ResNet50, (224, 224), 152, optimizer='adam', preprocessing=None, model_name_option='')

total layer count 175
Found 20787 images belonging to 10 classes.
Found 1637 images belonging to 10 classes.
Epoch 1/5
163/163 [==============================] - 324s - loss: 0.2577 - acc: 0.9191 - val_loss: 2.6435 - val_acc: 0.5504
Epoch 2/5
163/163 [==============================] - 265s - loss: 0.0574 - acc: 0.9825 - val_loss: 1.3095 - val_acc: 0.7495
Epoch 3/5
163/163 [==============================] - 267s - loss: 0.0394 - acc: 0.9873 - val_loss: 1.1674 - val_acc: 0.7337
Epoch 4/5
163/163 [==============================] - 266s - loss: 0.0348 - acc: 0.9893 - val_loss: 1.2046 - val_acc: 0.7385
Epoch 5/5
163/163 [==============================] - 266s - loss: 0.0239 - acc: 0.9925 - val_loss: 1.0548 - val_acc: 0.7428


In [4]:
run_a_model(InceptionV3, (299, 299), 172, optimizer='adam', preprocessing=inception_v3.preprocess_input, model_name_option='')

total layer count 311
Found 20787 images belonging to 10 classes.
Found 1637 images belonging to 10 classes.
Epoch 1/5
163/163 [==============================] - 509s - loss: 0.1854 - acc: 0.9427 - val_loss: 1.2199 - val_acc: 0.7575
Epoch 2/5
163/163 [==============================] - 493s - loss: 0.0370 - acc: 0.9897 - val_loss: 1.0980 - val_acc: 0.7764
Epoch 3/5
163/163 [==============================] - 493s - loss: 0.0214 - acc: 0.9945 - val_loss: 0.3628 - val_acc: 0.8968
Epoch 4/5
163/163 [==============================] - 493s - loss: 0.0192 - acc: 0.9952 - val_loss: 0.8447 - val_acc: 0.8155
Epoch 5/5
163/163 [==============================] - 493s - loss: 0.0202 - acc: 0.9940 - val_loss: 1.1769 - val_acc: 0.7795


In [5]:
run_a_model(Xception, (299, 299), 116, optimizer='adam', preprocessing=xception.preprocess_input, model_name_option='')

total layer count 132
Found 20787 images belonging to 10 classes.
Found 1637 images belonging to 10 classes.
Epoch 1/5
163/163 [==============================] - 627s - loss: 0.2350 - acc: 0.9277 - val_loss: 0.8607 - val_acc: 0.7019
Epoch 2/5
163/163 [==============================] - 621s - loss: 0.0284 - acc: 0.9912 - val_loss: 3.8232 - val_acc: 0.3519
Epoch 3/5
163/163 [==============================] - 621s - loss: 0.0155 - acc: 0.9958 - val_loss: 0.7173 - val_acc: 0.8296
Epoch 4/5
163/163 [==============================] - 621s - loss: 0.0112 - acc: 0.9966 - val_loss: 0.8089 - val_acc: 0.7764
Epoch 5/5
163/163 [==============================] - 621s - loss: 0.0134 - acc: 0.9962 - val_loss: 0.5681 - val_acc: 0.8363


In [10]:
# ResNet50 156 finetune Layer 10 epoches
run_a_model(ResNet50, (240, 360), 154, preprocessing=None, model_name_option='_154')

total layer count 175
Found 21601 images belonging to 10 classes.
Found 823 images belonging to 10 classes.
Epoch 1/5
169/169 [==============================] - 438s - loss: 0.2612 - acc: 0.9152 - val_loss: 1.2752 - val_acc: 0.6537
Epoch 2/5
169/169 [==============================] - 430s - loss: 0.0590 - acc: 0.9824 - val_loss: 0.7990 - val_acc: 0.8433
Epoch 3/5
169/169 [==============================] - 430s - loss: 0.0420 - acc: 0.9864 - val_loss: 1.2145 - val_acc: 0.7582
Epoch 4/5
169/169 [==============================] - 430s - loss: 0.0254 - acc: 0.9927 - val_loss: 1.4943 - val_acc: 0.6574
Epoch 5/5
169/169 [==============================] - 430s - loss: 0.0282 - acc: 0.9916 - val_loss: 0.9556 - val_acc: 0.7327
Epoch 1/5
169/169 [==============================] - 434s - loss: 0.0115 - acc: 0.9970 - val_loss: 0.7865 - val_acc: 0.7436
Epoch 2/5
169/169 [==============================] - 431s - loss: 0.0076 - acc: 0.9980 - val_loss: 0.6451 - val_acc: 0.7716
Epoch 3/5
169/169 [=====

基础预测

In [31]:
from keras.models import load_model
model = load_model('models/model_VGG16.h5')

In [33]:
gen = ImageDataGenerator()
path_test_data = 'dataset/to_prediction'
test_generator = gen.flow_from_directory(path_test_data,  (224, 224), shuffle=False, batch_size=batch_size, class_mode=None)

Found 79726 images belonging to 1 classes.


In [35]:
def load_test(img_rows, img_cols):
    print('Read test images')
    path = os.path.join(dataset_path, 'test', '*.jpg')
    files = glob.glob(path)
    X_test = []
    X_test_id = []
    total = 0
    thr = math.floor(len(files)/10)
    for fl in tqdm(files):
        flbase = os.path.basename(fl)
        img = get_img(fl, (img_rows, img_cols))
        X_test.append(img)
        X_test_id.append(flbase)
        total += 1

    return X_test, X_test_id


def make_predictions(MODEL, image_size, batch_size):
    gen = ImageDataGenerator()
    path_test_data = 'dataset/to_prediction'
    test_generator = gen.flow_from_directory(path_test_data,  image_size, shuffle=False, 
                                             batch_size=batch_size, class_mode=None)
    y_predictions = MODEL.predict_generator(test_generator,  steps=test_generator.samples//batch_size+1,  verbose=1)
    y_predictions = y_predictions.clip(min=0.005, max=0.995)
    
    test_id = list()
    for i, file_name in enumerate(test_generator.filenames):
        if i >5:
            break
        print(file_name)
        # name = fname[fname.rfind('/')+1:]
        # l.append( [name, *y_pred[i]] )
        
    
    return y_predictions

def create_submission(predictions, test_id):
    result1 = pd.DataFrame(predictions, columns=['c0', 'c1', 'c2', 'c3',
                                                 'c4', 'c5', 'c6', 'c7',
                                                 'c8', 'c9'])
    result1.loc[:, 'img'] = pd.Series(test_id, index=result1.index)
    now = datetime.datetime.now()
    if not os.path.isdir('subm'):
        os.mkdir('subm')
    suffix = str(now.strftime("%Y-%m-%d-%H-%M"))
    sub_file = os.path.join('subm', 'submission_' + suffix + '.csv')
    result1.to_csv(sub_file, index=False)

In [25]:
def gen_kaggle_csv(model,  model_image_size, csv_name):
    dir = "dataset/"

    gen = ImageDataGenerator()
    test_generator = gen.flow_from_directory(dir + "test/",  model_image_size, shuffle=False, 
                                             batch_size=batch_size, class_mode=None)
    y_pred = model.predict_generator(test_generator,  steps=test_generator.samples//batch_size+1,  verbose=1)
    print("y_pred shape {}".format(y_pred.shape))
    y_pred = y_pred.clip(min=0.005, max=0.995)
    print(y_pred[:3])


In [ ]:
# X_test_to_pred_test , X_test_id_test = load_test(224, 224)
X_test_to_pred = np.array(X_test_to_pred)
print("pred begin")
start = time.time()
y_pred = model.predict(X_test_to_pred, verbose=1)
end = time.time()
print(end - start)

In [114]:
create_submission(y_pred, X_test_id)